In [6]:
import torch
import torch.nn as nn
from src.models.model_structure import BasicBlock, customResNet18
import argparse
from src.utils.configer import Configer
from pathlib import Path

In [7]:
parser = argparse.ArgumentParser()

parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--hypes', default=None, type=str,
                    dest='hypes', help='The file of the hyper parameters.')
parser.add_argument('--phase', default='train', type=str,
                    dest='phase', help='The phase of module.')
parser.add_argument('--gpu', default=[0, ], nargs='+', type=int,
                    dest='gpu', help='The gpu used.')
parser.add_argument('--resume', default=None, type=str,
                    dest='resume', help='The path of pretrained model.')

args, unknown = parser.parse_known_args()


args.device = None
if not args.disable_cuda and torch.cuda.is_available():
    args.device = torch.device('cuda')
else:
    args.device = torch.device('cpu')


args.hypes = 'src/hyperparameters/config.json'
configer = Configer(args)

selected_classes = configer.get('selected_classes')
n_classes = len(selected_classes )

In [8]:
seed = 1991
torch.manual_seed(seed)
in_channels = 3
model = customResNet18(num_classes=n_classes)
test_input = torch.randn(2, in_channels, 64, 64)
model(test_input)

tensor([[-0.1056, -0.7922,  0.2552,  0.3541, -0.2861,  0.1664,  0.0494,  0.2417,
         -0.7143, -0.3899],
        [-0.0054, -0.6365,  0.4501,  0.2863, -0.1228, -0.1621,  0.3041,  0.0216,
         -0.6327, -0.3545]], grad_fn=<AddmmBackward0>)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Количество обучаемых параметров: {count_parameters(model):,}")

Количество обучаемых параметров: 2,788,496


In [10]:
from torch.utils.data import DataLoader
from src.datasets.TinyImageNetDataset import TinyImageNetDataset
import torchvision.transforms as transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
train_transforms = transforms.Compose([
                transforms.Resize((72, 72)),
                transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(10),
                transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
                transforms.ToTensor(),
                normalize,
            ])

train_dataset = DataLoader(
            TinyImageNetDataset(
                Path(configer.get('data', 'data_path')),
                split=configer.get('phase'),
                transform=train_transforms),
            batch_size=configer.get('data', 'batch_size'),
            shuffle=True,
            num_workers=configer.get('data', 'workers'))

In [13]:
device = torch.device(configer.get("device") if torch.cuda.is_available() else 'cpu')
from tqdm import tqdm
for data_tuple in tqdm(train_dataset, desc="Train"):
        inputs, gt = data_tuple[0].to(device), data_tuple[1].to(device)
        outputs = model(inputs)

Train: 100%|██████████| 782/782 [09:18<00:00,  1.40it/s]
